In [1]:
# for path in sys.path:
#   print(path)

import sys
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

import logging
import argparse # Argument parsing
import math
from collections import defaultdict
import concurrent
from concurrent.futures import ThreadPoolExecutor  # Importing specific executor for clarity
import time

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Nearest neighbor search
import sklearn
from sklearn.neighbors import BallTree

# Special functions and optimizations
from scipy.special import gamma, kv  # Bessel function and gamma function
from scipy.stats import multivariate_normal  # Simulation
from scipy.optimize import minimize
from scipy.spatial.distance import cdist  # For space and time distance
from scipy.spatial import distance  # Find closest spatial point
from scipy.optimize import differential_evolution

# Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Type hints
from typing import Callable, Union, Tuple

# Add your custom path
# sys.path.append("/cache/home/jl2815/tco")

# Custom imports

from GEMS_TCO import orbitmap 
from GEMS_TCO import kernels 
from GEMS_TCO import evaluate
from GEMS_TCO import orderings as _orderings

import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor

In [2]:
lat_lon_resolution = [15,15]
mm_cond_number = 10
params= [20, 8.25, 5.25, 0.2, 0.5, 5]
idx_for_datamap= [0,8]

In [3]:
# Load the one dictionary to set spaital coordinates
# filepath = "C:/Users/joonw/TCO/GEMS_data/data_2023/sparse_cen_map23_01.pkl"
filepath = "/Users/joonwonlee/Documents/GEMS_DATA/pickle_2023/coarse_cen_map23_01.pkl"
with open(filepath, 'rb') as pickle_file:
    coarse_dict_24_1 = pickle.load(pickle_file)

sample_df = coarse_dict_24_1['y23m01day01_hm02:12']

sample_key = coarse_dict_24_1.get('y23m01day01_hm02:12')
if sample_key is None:
    print("Key 'y23m01day01_hm02:12' not found in the dictionary.")

# { (20,20):(5,1), (5,5):(20,40) }
rho_lat = lat_lon_resolution[0]          
rho_lon = lat_lon_resolution[1]
lat_n = sample_df['Latitude'].unique()[::rho_lat]
lon_n = sample_df['Longitude'].unique()[::rho_lon]

lat_number = len(lat_n)
lon_number = len(lon_n)

# Set spatial coordinates for each dataset
coarse_dicts = {}

years = ['2024']
for year in years:
    for month in range(7, 8):  # Iterate over all months
        # filepath = f"C:/Users/joonw/TCO/GEMS_data/data_{year}/sparse_cen_map{year[2:]}_{month:02d}.pkl"
        filepath = f"/Users/joonwonlee/Documents/GEMS_DATA/pickle_{year}/coarse_cen_map{year[2:]}_{month:02d}.pkl"
        with open(filepath, 'rb') as pickle_file:
            loaded_map = pickle.load(pickle_file)
            for key in loaded_map:
                tmp_df = loaded_map[key]
                coarse_filter = (tmp_df['Latitude'].isin(lat_n)) & (tmp_df['Longitude'].isin(lon_n))
                coarse_dicts[f"{year}_{month:02d}_{key}"] = tmp_df[coarse_filter].reset_index(drop=True)


key_idx = sorted(coarse_dicts)
if not key_idx:
    raise ValueError("coarse_dicts is empty")

# extract first hour data because all data shares the same spatial grid
data_for_coord = coarse_dicts[key_idx[0]]
x1 = data_for_coord['Longitude'].values
y1 = data_for_coord['Latitude'].values 
coords1 = np.stack((x1, y1), axis=-1)


# instance = orbitmap.MakeOrbitdata(data_for_coord, lat_s=5, lat_e=10, lon_s=110, lon_e=120)
# s_dist = cdist(coords1, coords1, 'euclidean')
# ord_mm, _ = instance.maxmin_naive(s_dist, 0)

ord_mm = _orderings.maxmin_cpp(coords1)
data_for_coord = data_for_coord.iloc[ord_mm].reset_index(drop=True)
coords1_reordered = np.stack((data_for_coord['Longitude'].values, data_for_coord['Latitude'].values), axis=-1)
# nns_map = instance.find_nns_naive(locs=coords1_reordered, dist_fun='euclidean', max_nn=mm_cond_number)
nns_map=_orderings.find_nns_l2(locs= coords1_reordered  ,max_nn = mm_cond_number)


analysis_data_map = {}
for i in range(idx_for_datamap[0],idx_for_datamap[1]):
    tmp = coarse_dicts[key_idx[i]].copy()
    tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)

    tmp = tmp.iloc[ord_mm, :4].to_numpy()
    tmp = torch.from_numpy(tmp).float()  # Convert NumPy to Tensor
    # tmp = tmp.clone().detach().requires_grad_(True)  # Enable gradients
    
    analysis_data_map[key_idx[i]] = tmp

aggregated_data = pd.DataFrame()
for i in range(idx_for_datamap[0],idx_for_datamap[1]):
    tmp = coarse_dicts[key_idx[i]].copy()
    tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)
    tmp = tmp.iloc[ord_mm].reset_index(drop=True)  
    aggregated_data = pd.concat((aggregated_data, tmp), axis=0)

aggregated_data = aggregated_data.iloc[:, :4].to_numpy()

aggregated_data = torch.from_numpy(aggregated_data).float()  # Convert NumPy to Tensor
# aggregated_np = aggregated_np.clone().detach().requires_grad_(True)  # Enable gradients




In [4]:
result = {}

for day in range(2):
    idx_for_datamap= [8*day,8*(day+1)]
    analysis_data_map = {}
    for i in range(idx_for_datamap[0],idx_for_datamap[1]):
        tmp = coarse_dicts[key_idx[i]].copy()
        tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)

        tmp = tmp.iloc[ord_mm, :4].to_numpy()
        tmp = torch.from_numpy(tmp).float()  # Convert NumPy to Tensor
        # tmp = tmp.clone().detach().requires_grad_(True)  # Enable gradients
        
        analysis_data_map[key_idx[i]] = tmp
    aggregated_data = pd.DataFrame()
    for i in range(idx_for_datamap[0],idx_for_datamap[1]):
        tmp = coarse_dicts[key_idx[i]].copy()
        tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)
        tmp = tmp.iloc[ord_mm].reset_index(drop=True)  
        aggregated_data = pd.concat((aggregated_data, tmp), axis=0)
    
    aggregated_data = aggregated_data.iloc[:, :4].to_numpy()

    aggregated_data = torch.from_numpy(aggregated_data).float()  # Convert NumPy to Tensor

    params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
    params = torch.tensor(params, requires_grad=True)

    torch_smooth = torch.tensor(0.5, dtype=torch.float32)


    instance = kernels.model_fitting(
        smooth=0.5,
        input_map=analysis_data_map,
        aggregated_data=aggregated_data,
        nns_map=nns_map,
        mm_cond_number=mm_cond_number
    )

    # optimizer = optim.Adam([params], lr=0.01)  # For Adam
    optimizer, scheduler = instance.optimizer_fun(params, lr=0.01, betas=(0.9, 0.8), eps=1e-8, step_size=20, gamma=0.9)    
    out = instance.run_full(params, optimizer,scheduler, epochs=3000)
    result[day+1] = out
        

Epoch 1, Gradients: [ -1.3326397   8.336441    8.247559    1.1887207 -22.827812  156.88403
  -4.3982744]
 Loss: 1392.27001953125, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]
Epoch 101, Gradients: [-1.0859816   0.16093826 -0.13684082 -0.5060425   0.31221008 -2.2960205
 -4.613434  ]
 Loss: 1378.829833984375, Parameters: [ 2.5250835e+01  2.0382962e+00  2.0210776e+00  2.7965151e-02
 -9.0995645e-03  1.4091690e-01  4.1661091e+00]
Epoch 201, Gradients: [-0.9023218  -0.01376152 -0.01701355 -0.07843018  0.14977264 -0.69732666
 -3.442699  ]
 Loss: 1376.39404296875, Parameters: [ 2.5738956e+01  2.2357175e+00  2.2231426e+00  2.7480897e-02
 -1.6363824e-02  1.2432123e-01  4.6563659e+00]
Epoch 301, Gradients: [-8.1594217e-01  1.2187958e-03 -7.5073242e-03  3.0197144e-02
 -4.5982361e-02  1.1151123e-01 -2.7896533e+00]
 Loss: 1375.2481689453125, Parameters: [ 2.60260925e+01  2.36984658e+00  2.35182714e+00  2.70866435e-02
 -2.03116890e-02  1.15547456e-01  4.

In [ ]:
isinstance(result, dict)
import os
# Save pickle
output_filename = f"estimation_1250_july24.pkl"

# base_path = "/home/jl2815/tco/data/pickle_data"
output_path = "/Users/joonwonlee/Documents/"
output_filepath = os.path.join(output_path, output_filename)
with open(output_filepath, 'wb') as pickle_file:
    pickle.dump(result, pickle_file)

input_filepath = output_filepath
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    loaded_map = pickle.load(pickle_file)

loaded_map

{1: [array([ 2.6412588e+01,  2.5630600e+00,  2.5388060e+00,  2.7070900e-02,
         -2.4097290e-02,  1.0483751e-01,  5.3329625e+00], dtype=float32),
  1374.03466796875],
 2: [array([24.588697  ,  3.3285947 ,  3.9052231 ,  0.13455719, -0.03456095,
          0.0597389 ,  2.938084  ], dtype=float32),
  1148.790771484375]}